In [1]:
import random
import operator
import networkx as nx
from time import *
from networkx.algorithms import connectivity
from networkx.algorithms import complete_to_chordal_graph, chordal_graph_cliques

In [2]:
def complete_chordal_graph(g):
    H = g.copy()
    alpha = {node: 0 for node in H}
    chords = set()
    weight = {node: 0 for node in H.nodes()}
    unnumbered_nodes = list(H.nodes())
    for i in range(len(H.nodes()), 0, -1):
        # get the node in unnumbered_nodes with the maximum weight
        z = max(unnumbered_nodes, key=lambda node: weight[node])
        unnumbered_nodes.remove(z)
        alpha[z] = i
        update_nodes = []
        for y in unnumbered_nodes:
            if g.has_edge(y, z):
                update_nodes.append(y)
            else:
                # y_weight will be bigger than node weights between y and z
                y_weight = weight[y]
                lower_nodes = [
                    node for node in unnumbered_nodes if weight[node] < y_weight
                ]
                if nx.has_path(H.subgraph(lower_nodes + [z, y]), y, z):
                    update_nodes.append(y)
                    chords.add((z, y))
        # during calculation of paths the weights should not be updated
        for node in update_nodes:
            weight[node] += 1
    H.add_edges_from(chords)
    return H, alpha,weight
def F_t(g):
    H, alpha,weight = complete_chordal_graph(g)
    f_t = []
    alpha_up = sorted(alpha, key=alpha.__getitem__)
    for i in range(len(list(g.nodes))-1,0,-1):
        if weight[alpha_up[i-1]] <= weight[alpha_up[i]]:
            f_t.append(alpha_up[i-1])
    return H,f_t,alpha_up
def p_decomposition(gg):
    CH,F_a, pai_2 = F_t(gg)
    prime_block = []
    block_A = []
    block_B = []
    S_ft = []
    g=gg.copy()
    for i in range(len(F_a)-1,-1,-1):    
        S_ft = list(CH.adj[F_a[i]])
        S_ft = [s for s in g.nodes if s in S_ft]
        if len(S_ft) == 0:
            prime_block.append(F_a[i])
            V = [s for s in g.nodes if s != F_a[i]]
            g = nx.subgraph(g, V)          
        mm = 1
        while mm>0:
            mm = 0
            for j in range(0,len(S_ft)):
                if pai_2.index(S_ft[j]) < pai_2.index(F_a[i]):
                    S_ft.remove(S_ft[j])
                    mm = 1
                    break          
        if len(S_ft) == 1:
            V_remove_S_ft = [q for q in g.nodes if q not in S_ft]
            M_c=list(nx.connected_components(nx.subgraph(g, V_remove_S_ft)))
            for k in range(0,len(M_c)):
                if F_a[i] in list(M_c[k]):
                    brock_A = list(M_c[k])
                    brock_A.extend(S_ft)
                    block_B = [p for p in g.nodes if p not in list(M_c[k])]
                    prime_block.append(brock_A)
                    g = nx.subgraph(g, block_B)
        if len(S_ft) > 1:
            S_ft_subgraph = g.subgraph(S_ft)
            n_S_ft = len(S_ft_subgraph.nodes)
            m_S_ft = len(S_ft_subgraph.edges)
            m_max_S_ft = int((n_S_ft * (n_S_ft - 1))/2) 
            if m_S_ft == m_max_S_ft:
                V_remove_S_ft = [t for t in g.nodes if t not in S_ft]
                M_c=list(nx.connected_components(nx.subgraph(g, V_remove_S_ft)))
                for k in range(0,len(M_c)):
                    if F_a[i] in list(M_c[k]):
                        block_A = list(M_c[k])
                        block_A.extend(S_ft)
                        block_B = [p for p in g.nodes if p not in list(M_c[k])]
                        prime_block.append(block_A)
                        g = nx.subgraph(g, block_B)
    M_g=list(nx.connected_components(g))
    for i in M_g:
        prime_block.append(list(i))
    for i in prime_block:
        if isinstance (i,list)==False:
            K = []
            K.append(i)
            prime_block[prime_block.index(i)] = K
    return prime_block

def MSF(r,g):
    prime_block = p_decomposition(g)
    K_list=[]
    h = []
    h.extend(r)
    # Pruning procedure
    for i in range(0,len(prime_block)):
        s = set()
        for j in range(0,len(prime_block)):
            if j!=i:
                s = s | set(prime_block[j])
        K_list.append(s & set(prime_block[i]))

    p=0
    while p == 0:
        if len(prime_block) == 1:
            break
        p = 1
        for i in range(0,len(prime_block)):
            if len((set(prime_block[i]) - K_list[i]) &  set(r)) == 0:
                for j in range(0,len(prime_block)):
                    if j!=i:
                        if len(K_list[i] - (set(prime_block[i]) & set(prime_block[j]))) == 0:
                            del prime_block[i]
                            K_list=[]
                            for k in range(0,len(prime_block)):
                                s = set()
                                for l in range(0,len(prime_block)):
                                    if l!=k:
                                        s = s | set(prime_block[l])
                                K_list.append(s & set(prime_block[k]))
                            p = p-1
                            break
            if p == 1 and i < (len(prime_block)-1):
                continue   
            break

                
    # Slimming procedure      
    for i in range(len(prime_block)-1,-1,-1):
        if len((set(r) & set(prime_block[i])) - K_list[i]) == 0:
            sg = g.subgraph(list(K_list[i]))
            n_sg = len(sg.nodes)
            m_sg = len(sg.edges)
            m_max = int((n_sg * (n_sg - 1))/2) 
            if m_sg == m_max:
                h.extend(list(K_list[i]))                
            else:
                h.extend(list(prime_block[i]))
        else:
            sg = g.subgraph(list((set(r) & set(prime_block[i]))| K_list[i]))
            n_sg = len(sg.nodes)
            m_sg = len(sg.edges)
            m_max = int((n_sg * (n_sg - 1))/2) 
            if m_sg == m_max:
                h.extend(list((set(r) & set(prime_block[i]))| K_list[i]))
            else:
                h.extend(list(set(prime_block[i])))
    h_de = list(set(h))
    return h_de

In [3]:
G= nx.random_graphs.fast_gnp_random_graph(100,  0.05,  seed=None, directed=False)
R = random.sample(list(G.nodes), 20)
H_MSF = MSF(R, G)